# Map Reduce

**MapReduce** is a programming model for performing parallel processing on large datasets. Although it is a powerful technique, its basics are relatively simple.

Imagine we have a collection of items we’d like to process somehow. For instance, the items might be website logs, the texts of various books, image files, or anything else. A basic version of the MapReduce algorithm consists of the following steps:

1. Use a `mapper function` to turn each item into zero or more key/value pairs. (Often this is called the map function, but there is already a Python function called map and we don’t need to confuse the two.)
2. Collect together all the pairs with identical keys.
3. Use a `reducer function` on each collection of grouped values to produce output values for the corresponding key.

The primary benefit of **MapReduce** is that it allows us to distribute computations by moving the processing to the data. Imagine we want to word-count across billions of documents.

Non-MapReduce approach requires the machine doing the processing to have access to every document. This means that the documents all need to either live on that machine or else be transferred to it during processing. More important, it means that the machine can process only one document at a time. 

## Example 1: Word Count
Find the most frequent words.

In [1]:
# Non Map Reduce version
from typing import List
from collections import Counter

def tokenize(document: str) -> List[str]:
    """Just split on whitespace"""
    return document.split()

def word_count_old(documents: List[str]):
    """Word count not using MapReduce"""
    return Counter(word
        for document in documents
        for word in tokenize(document)) 

With millions of users the set of documents (status updates) is suddenly too big to fit on your computer. If you can just fit this into the MapReduce model, you can use some “big data” infrastructure.

First, we need a `function that turns a document into a sequence of key/value pairs`. **We’ll want our output to be grouped by word, which means that the keys should be words**. And for each word, we’ll just emit the value 1 to indicate that this pair corresponds to one occurrence of the word. 

In [2]:
from typing import Iterator, Tuple

def wc_mapper(document: str) -> Iterator[Tuple[str, int]]:
    """For each word in the document, emit (word, 1)"""
    for word in tokenize(document):
        yield (word, 1) 

Skipping the “plumbing” step 2 for the moment, imagine that for some word we’ve collected a list of the corresponding counts we emitted. To produce the overall count for that word, then, we just need:

In [3]:
from typing import Iterable
def wc_reducer(word: str,
               counts: Iterable[int]) -> Iterator[Tuple[str, int]]:
    """Sum up the counts for a word"""
    yield (word, sum(counts))

Returning to step 2, we now need to collect the results from `wc_mapper` and feed them to `wc_reducer`.

In [4]:
from collections import defaultdict

def word_count(documents: List[str]) -> List[Tuple[str, int]]:
    """Count the words in the input documents using
    “MapReduce"""

    collector = defaultdict(list)  # To store grouped values

    for document in documents:
        for word, count in wc_mapper(document):
            collector[word].append(count)

    return [output
            for word, counts in collector.items()
            for output in wc_reducer(word, counts)]

In [5]:
documents = ["data science", "big data", "science fiction"]

In [6]:
word_count(documents)

[('data', 2), ('science', 2), ('big', 1), ('fiction', 1)]

Imagine now that our billions of documents are scattered across 100 machines. With the right infrastructure (and glossing over some of the details), we can do the following:

- Have each machine run the mapper on its documents, producing lots of key/value pairs.
- Distribute those key/value pairs to a number of “reducing” machines, making sure that the pairs corresponding to any given key all end up on the same machine.
- Have each reducing machine group the pairs by key and then run the reducer on each set of values.
- Return each (key, output) pair.

`What is amazing about this is that it scales horizontally`. If we double the number of machines, then (ignoring certain fixed costs of running a MapReduce system) our computation should run approximately twice as fast.